## Import libraries

In [15]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import glob
import requests
import holidays
import joblib

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Input,LSTM, Dense, Dropout

## Load and Prepare Data

In [3]:
# Path the local folder
data_folder = '../../data'

# Find all CSVs starting with 'vehicle_accident' in that folder
file_list = glob.glob(os.path.join(data_folder, 'vehicle_accident *.csv'))

# Load all files into a list of DataFrames
dfs = []
for file in file_list:
    df = pd.read_csv(file, encoding='utf-8', low_memory=False)
    dfs.append(df)

# Concatenate all DataFrames
df = pd.concat(dfs, ignore_index=True)
df.head(10)

,ปีที่เกิดเหตุ,วันที่เกิดเหตุ,เวลา,วันที่รายงาน,เวลาที่รายงาน,ACC_CODE,หน่วยงาน,สายทางหน่วยงาน,รหัสสายทาง,สายทาง,...,รถบรรทุก 6 ล้อ,รถบรรทุกมากกว่า 6 ล้อ ไม่เกิน 10 ล้อ,รถบรรทุกมากกว่า 10 ล้อ (รถพ่วง),รถอีแต๋น,อื่นๆ,คนเดินเท้า,จำนวนผู้เสียชีวิต,จำนวนผู้บาดเจ็บสาหัส,จำนวนผู้บาดเจ็บเล็กน้อย,รวมจำนวนผู้บาดเจ็บ
0,2022,01/01/2022,00:01,02/01/2022,11:45,6566872,กรมทางหลวงชนบท,ทางหลวงชนบท,ชน.5016,เทศบาลตำบลวัดสิงห์ - บ้านน้ำพุ (ช่วงหันคา),...,0,0,0,0,0,0,0,1,0,1
1,2022,01/01/2022,00:01,02/01/2022,11:44,6566880,กรมทางหลวงชนบท,ทางหลวงชนบท,มค.4012,แยกทางหลวงหมายเลข 2152 (กม.ที่ 31+700) - บ้านก...,...,0,0,0,0,0,0,0,0,1,1
2,2022,01/01/2022,00:03,09/02/2022,08:41,5706553,กรมทางหลวง,ทางหลวง,4,พ่อตาหินช้าง - วังครก,...,0,0,0,0,0,0,1,0,0,0
3,2022,01/01/2022,00:05,02/01/2022,06:21,5485750,กรมทางหลวง,ทางหลวง,4030,ถลาง - หาดราไวย์,...,0,0,0,0,0,0,0,0,1,1
4,2022,01/01/2022,00:05,24/01/2022,09:59,5624452,กรมทางหลวง,ทางหลวง,216,ถนนวงแหวนรอบเมืองอุดรธานีด้านทิศตะวันออก,...,0,0,0,0,0,0,0,0,2,2
5,2022,01/01/2022,00:05,02/01/2022,11:46,6566842,กรมทางหลวงชนบท,ทางหลวงชนบท,อย.4009,แยกทางหลวงหมายเลข 3111 (กม.ที่ 19+200) - บ้านป...,...,0,0,0,0,0,0,1,0,0,0
6,2022,01/01/2022,00:08,03/03/2022,10:14,5836781,กรมทางหลวง,ทางหลวง,3477,บางปะอิน - เกาะเรียน,...,0,0,0,0,0,0,0,0,1,1
7,2022,01/01/2022,00:10,22/02/2022,14:01,5783258,กรมทางหลวง,ทางหลวง,3438,ดินแดง - ไผ่งาม,...,0,0,0,0,0,0,0,1,0,1
8,2022,01/01/2022,00:10,02/01/2022,06:45,6566818,กรมทางหลวงชนบท,ทางหลวงชนบท,ชม.3059,แยกทางหลวงหมายเลข 107 (กม.ที่ 152+300) - เขื่อ...,...,0,0,0,0,0,0,0,0,2,2
9,2022,01/01/2022,00:18,02/01/2022,06:44,6566847,กรมทางหลวงชนบท,ทางหลวงชนบท,พง.5012,เชื่อมถนนเทศบาลท้ายเหมือง - ชายทะเลท้ายเหมือง,...,0,0,0,0,1,0,0,1,0,1


In [4]:
# Convert "วันที่เกิดเหตุ" and "เวลา" to datetime
df['datetime'] = pd.to_datetime(df['วันที่เกิดเหตุ'] + ' ' + df['เวลา'], format='%d/%m/%Y %H:%M')
df.dropna(subset=['LATITUDE', 'LONGITUDE', 'datetime','สภาพอากาศ'], inplace=True)
df

,ปีที่เกิดเหตุ,วันที่เกิดเหตุ,เวลา,วันที่รายงาน,เวลาที่รายงาน,ACC_CODE,หน่วยงาน,สายทางหน่วยงาน,รหัสสายทาง,สายทาง,...,รถบรรทุกมากกว่า 6 ล้อ ไม่เกิน 10 ล้อ,รถบรรทุกมากกว่า 10 ล้อ (รถพ่วง),รถอีแต๋น,อื่นๆ,คนเดินเท้า,จำนวนผู้เสียชีวิต,จำนวนผู้บาดเจ็บสาหัส,จำนวนผู้บาดเจ็บเล็กน้อย,รวมจำนวนผู้บาดเจ็บ,datetime
0,2022,01/01/2022,00:01,02/01/2022,11:45,6566872,กรมทางหลวงชนบท,ทางหลวงชนบท,ชน.5016,เทศบาลตำบลวัดสิงห์ - บ้านน้ำพุ (ช่วงหันคา),...,0,0,0,0,0,0,1,0,1,2022-01-01 00:01:00
1,2022,01/01/2022,00:01,02/01/2022,11:44,6566880,กรมทางหลวงชนบท,ทางหลวงชนบท,มค.4012,แยกทางหลวงหมายเลข 2152 (กม.ที่ 31+700) - บ้านก...,...,0,0,0,0,0,0,0,1,1,2022-01-01 00:01:00
2,2022,01/01/2022,00:03,09/02/2022,08:41,5706553,กรมทางหลวง,ทางหลวง,4,พ่อตาหินช้าง - วังครก,...,0,0,0,0,0,1,0,0,0,2022-01-01 00:03:00
3,2022,01/01/2022,00:05,02/01/2022,06:21,5485750,กรมทางหลวง,ทางหลวง,4030,ถลาง - หาดราไวย์,...,0,0,0,0,0,0,0,1,1,2022-01-01 00:05:00
4,2022,01/01/2022,00:05,24/01/2022,09:59,5624452,กรมทางหลวง,ทางหลวง,216,ถนนวงแหวนรอบเมืองอุดรธานีด้านทิศตะวันออก,...,0,0,0,0,0,0,0,2,2,2022-01-01 00:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226105,2013,31/12/2013,23:00,13/01/2014,13:33,3576908,กรมทางหลวง,ทางหลวง,9,NaN,...,0,0,0,0,0,0,0,0,0,2013-12-31 23:00:00
226106,2013,31/12/2013,23:00,04/02/2014,12:08,3578962,กรมทางหลวง,ทางหลวง,41,สวนสมบูรณ์ - เกาะมุกข์,...,0,0,0,0,0,0,1,0,1,2013-12-31 23:00:00
226107,2013,31/12/2013,23:00,04/02/2014,12:29,3579135,กรมทางหลวง,ทางหลวง,202,แก้งสนามนาง - ดอนตะหนิน,...,0,0,0,0,0,1,0,0,0,2013-12-31 23:00:00
226108,2013,31/12/2013,23:25,06/01/2014,11:44,3576181,กรมทางหลวง,ทางหลวง,4170,สระเกศ - หัวถนน,...,0,0,0,0,0,0,1,0,1,2013-12-31 23:25:00


In [5]:
# Round down the minute to make it a flat hour store as new column named "datetime_hour"
df['datetime_hour'] = df['datetime'].dt.floor('h')
df

,ปีที่เกิดเหตุ,วันที่เกิดเหตุ,เวลา,วันที่รายงาน,เวลาที่รายงาน,ACC_CODE,หน่วยงาน,สายทางหน่วยงาน,รหัสสายทาง,สายทาง,...,รถบรรทุกมากกว่า 10 ล้อ (รถพ่วง),รถอีแต๋น,อื่นๆ,คนเดินเท้า,จำนวนผู้เสียชีวิต,จำนวนผู้บาดเจ็บสาหัส,จำนวนผู้บาดเจ็บเล็กน้อย,รวมจำนวนผู้บาดเจ็บ,datetime,datetime_hour
0,2022,01/01/2022,00:01,02/01/2022,11:45,6566872,กรมทางหลวงชนบท,ทางหลวงชนบท,ชน.5016,เทศบาลตำบลวัดสิงห์ - บ้านน้ำพุ (ช่วงหันคา),...,0,0,0,0,0,1,0,1,2022-01-01 00:01:00,2022-01-01 00:00:00
1,2022,01/01/2022,00:01,02/01/2022,11:44,6566880,กรมทางหลวงชนบท,ทางหลวงชนบท,มค.4012,แยกทางหลวงหมายเลข 2152 (กม.ที่ 31+700) - บ้านก...,...,0,0,0,0,0,0,1,1,2022-01-01 00:01:00,2022-01-01 00:00:00
2,2022,01/01/2022,00:03,09/02/2022,08:41,5706553,กรมทางหลวง,ทางหลวง,4,พ่อตาหินช้าง - วังครก,...,0,0,0,0,1,0,0,0,2022-01-01 00:03:00,2022-01-01 00:00:00
3,2022,01/01/2022,00:05,02/01/2022,06:21,5485750,กรมทางหลวง,ทางหลวง,4030,ถลาง - หาดราไวย์,...,0,0,0,0,0,0,1,1,2022-01-01 00:05:00,2022-01-01 00:00:00
4,2022,01/01/2022,00:05,24/01/2022,09:59,5624452,กรมทางหลวง,ทางหลวง,216,ถนนวงแหวนรอบเมืองอุดรธานีด้านทิศตะวันออก,...,0,0,0,0,0,0,2,2,2022-01-01 00:05:00,2022-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226105,2013,31/12/2013,23:00,13/01/2014,13:33,3576908,กรมทางหลวง,ทางหลวง,9,NaN,...,0,0,0,0,0,0,0,0,2013-12-31 23:00:00,2013-12-31 23:00:00
226106,2013,31/12/2013,23:00,04/02/2014,12:08,3578962,กรมทางหลวง,ทางหลวง,41,สวนสมบูรณ์ - เกาะมุกข์,...,0,0,0,0,0,1,0,1,2013-12-31 23:00:00,2013-12-31 23:00:00
226107,2013,31/12/2013,23:00,04/02/2014,12:29,3579135,กรมทางหลวง,ทางหลวง,202,แก้งสนามนาง - ดอนตะหนิน,...,0,0,0,0,1,0,0,0,2013-12-31 23:00:00,2013-12-31 23:00:00
226108,2013,31/12/2013,23:25,06/01/2014,11:44,3576181,กรมทางหลวง,ทางหลวง,4170,สระเกศ - หัวถนน,...,0,0,0,0,0,1,0,1,2013-12-31 23:25:00,2013-12-31 23:00:00


In [6]:
# One-hot encode "สภาพอากาศ"
weather_dummies = pd.get_dummies(df['สภาพอากาศ'], prefix='weather')
df = pd.concat([df, weather_dummies], axis=1)
df = df.drop('สภาพอากาศ', axis=1)
df

,ปีที่เกิดเหตุ,วันที่เกิดเหตุ,เวลา,วันที่รายงาน,เวลาที่รายงาน,ACC_CODE,หน่วยงาน,สายทางหน่วยงาน,รหัสสายทาง,สายทาง,...,รวมจำนวนผู้บาดเจ็บ,datetime,datetime_hour,weather_ดินถล่ม,weather_ฝนตก,weather_ภัยธรรมชาติ เช่น พายุ น้ำท่วม,weather_มีหมอก/ควัน/ฝุ่น,weather_มืดครึ้ม,weather_อื่นๆ,weather_แจ่มใส
0,2022,01/01/2022,00:01,02/01/2022,11:45,6566872,กรมทางหลวงชนบท,ทางหลวงชนบท,ชน.5016,เทศบาลตำบลวัดสิงห์ - บ้านน้ำพุ (ช่วงหันคา),...,1,2022-01-01 00:01:00,2022-01-01 00:00:00,False,False,False,False,False,False,True
1,2022,01/01/2022,00:01,02/01/2022,11:44,6566880,กรมทางหลวงชนบท,ทางหลวงชนบท,มค.4012,แยกทางหลวงหมายเลข 2152 (กม.ที่ 31+700) - บ้านก...,...,1,2022-01-01 00:01:00,2022-01-01 00:00:00,False,False,False,False,True,False,False
2,2022,01/01/2022,00:03,09/02/2022,08:41,5706553,กรมทางหลวง,ทางหลวง,4,พ่อตาหินช้าง - วังครก,...,0,2022-01-01 00:03:00,2022-01-01 00:00:00,False,False,False,False,False,False,True
3,2022,01/01/2022,00:05,02/01/2022,06:21,5485750,กรมทางหลวง,ทางหลวง,4030,ถลาง - หาดราไวย์,...,1,2022-01-01 00:05:00,2022-01-01 00:00:00,False,False,False,False,False,False,True
4,2022,01/01/2022,00:05,24/01/2022,09:59,5624452,กรมทางหลวง,ทางหลวง,216,ถนนวงแหวนรอบเมืองอุดรธานีด้านทิศตะวันออก,...,2,2022-01-01 00:05:00,2022-01-01 00:00:00,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226105,2013,31/12/2013,23:00,13/01/2014,13:33,3576908,กรมทางหลวง,ทางหลวง,9,NaN,...,0,2013-12-31 23:00:00,2013-12-31 23:00:00,False,False,False,False,False,False,True
226106,2013,31/12/2013,23:00,04/02/2014,12:08,3578962,กรมทางหลวง,ทางหลวง,41,สวนสมบูรณ์ - เกาะมุกข์,...,1,2013-12-31 23:00:00,2013-12-31 23:00:00,False,False,False,False,False,False,True
226107,2013,31/12/2013,23:00,04/02/2014,12:29,3579135,กรมทางหลวง,ทางหลวง,202,แก้งสนามนาง - ดอนตะหนิน,...,0,2013-12-31 23:00:00,2013-12-31 23:00:00,False,False,False,True,False,False,False
226108,2013,31/12/2013,23:25,06/01/2014,11:44,3576181,กรมทางหลวง,ทางหลวง,4170,สระเกศ - หัวถนน,...,1,2013-12-31 23:25:00,2013-12-31 23:00:00,False,False,False,False,False,False,True


In [7]:
weather_cols = [col for col in df.columns if col.startswith('weather_')]
# Seperate each lat and lon into a Zone
LAT_GRID_SIZE = 0.1
LON_GRID_SIZE = 0.1
df['lat_zone'] = (df['LATITUDE'] // LAT_GRID_SIZE).astype(int)
df['lon_zone'] = (df['LONGITUDE'] // LON_GRID_SIZE).astype(int)
df['zone_id'] = df['lat_zone'].astype(str) + '_' + df['lon_zone'].astype(str)

# Group by zone, datetime_hour and weather one-hot encoding and mark it as an accident reccord (accident = 1)
df_grouped = df.groupby(['zone_id', 'datetime_hour'] + weather_cols).size().reset_index(name='accident_count')
df_grouped['accident'] = 1
df_grouped.head(5)

,zone_id,datetime_hour,weather_ดินถล่ม,weather_ฝนตก,weather_ภัยธรรมชาติ เช่น พายุ น้ำท่วม,weather_มีหมอก/ควัน/ฝุ่น,weather_มืดครึ้ม,weather_อื่นๆ,weather_แจ่มใส,accident_count,accident
0,-1_180,2019-12-29 03:00:00,False,False,False,True,False,False,False,1,1
1,100_986,2013-02-10 17:00:00,False,False,False,False,False,False,True,1,1
2,100_986,2013-04-13 01:00:00,False,True,False,False,False,False,False,1,1
3,100_986,2013-04-17 10:00:00,False,True,False,False,False,False,False,1,1
4,100_986,2013-04-29 15:00:00,False,False,False,False,False,False,True,1,1


In [8]:
# Add back location and datetime features
df_grouped['LATITUDE'] = df_grouped['zone_id'].apply(lambda x: (int(x.split('_')[0]) + 0.5) * LAT_GRID_SIZE)
df_grouped['LONGITUDE'] = df_grouped['zone_id'].apply(lambda x: (int(x.split('_')[1]) + 0.5) * LON_GRID_SIZE)
df_grouped['hour'] = df_grouped['datetime_hour'].dt.hour
df_grouped['dayofweek'] = df_grouped['datetime_hour'].dt.dayofweek
df_grouped['month'] = df_grouped['datetime_hour'].dt.month

## Create Negative Samples

In [9]:
number_to_random = len(df_grouped)
rng = np.random.default_rng(seed=42)

negative_df = df_grouped.sample(n=number_to_random, replace=True).copy()
negative_df['datetime_hour'] = negative_df['datetime_hour'] + pd.to_timedelta(rng.integers(1, 1000, size=number_to_random), unit='h')
negative_df['accident'] = 0
negative_df['accident_count'] = 0

In [10]:
# Extract datetime features for negative samples
negative_df['LATITUDE'] = negative_df['zone_id'].apply(lambda x: (int(x.split('_')[0]) + 0.5) * LAT_GRID_SIZE)
negative_df['LONGITUDE'] = negative_df['zone_id'].apply(lambda x: (int(x.split('_')[1]) + 0.5) * LON_GRID_SIZE)
negative_df['hour'] = negative_df['datetime_hour'].dt.hour
negative_df['dayofweek'] = negative_df['datetime_hour'].dt.dayofweek
negative_df['month'] = negative_df['datetime_hour'].dt.month

## Merge and Preprocessing Data

In [11]:
# Merge postive and negative samples
merged_df = pd.concat([df_grouped, negative_df], ignore_index=True).sample(frac=1).reset_index(drop=True)

# Select a partial zone base on high number of accident
zone_number = merged_df['zone_id'].value_counts()
danger_zone = zone_number[zone_number > 50].index
merged_df = merged_df[merged_df['zone_id'].isin(danger_zone)].copy()

merged_df.sort_values(['zone_id', 'datetime_hour'], inplace=True)
merged_df['accident_lag1'] = merged_df.groupby('zone_id')['accident'].shift(1).fillna(0)
merged_df['rolling_mean_3'] = merged_df.groupby('zone_id')['accident'].rolling(3).mean().reset_index(0, drop=True).fillna(0)

# Normalize feature
weather_features = [col for col in merged_df.columns if col.startswith('weather_')]
features = ['LATITUDE', 'LONGITUDE', 'hour', 'dayofweek', 'month',
            'accident_lag1', 'rolling_mean_3'] + weather_features

scaler = MinMaxScaler()
merged_df[features] = scaler.fit_transform(merged_df[features])

## Create Time Based Sequences for LSTM

In [12]:
def create_sequences(df, time_steps=10):
    df = df.sort_values('datetime_hour')
    X = df[features].values
    y = df['accident'].values
    X_list, y_list = [], []
    for i in range(len(X) - time_steps):
        X_list.append(X[i:i+time_steps])
        y_list.append(y[i+time_steps])
    return X_list, y_list

time_steps = 10
X_seq_all = []
y_seq_all = []
for zone_id in danger_zone:
    zone_df = merged_df[merged_df['zone_id'] == zone_id]
    X_zone, y_zone = create_sequences(zone_df, time_steps=10)
    X_seq_all.extend(X_zone)
    y_seq_all.extend(y_zone)

X_seq = np.array(X_seq_all)
y_seq = np.array(y_seq_all)

## Build and Train LSTM Model

In [13]:
# Split train test data
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Define a class weight
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {0: weights[0], 1: weights[1]}

In [16]:
# Build the LSTM Model
model = Sequential([
    Input(shape=(X_seq.shape[1], X_seq.shape[2])),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 64)         │        20,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,673 (127.63 KB)

 Trainable params: 32,673 (127.63 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Train the LSTM Model
train = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/20
4904/4904 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.5153 - loss: 0.6926 - val_accuracy: 0.5182 - val_loss: 0.6917
Epoch 2/20
4904/4904 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.5250 - loss: 0.6915 - val_accuracy: 0.5222 - val_loss: 0.6916
Epoch 3/20
4904/4904 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.5249 - loss: 0.6913 - val_accuracy: 0.5275 - val_loss: 0.6905
Epoch 4/20
4904/4904 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.5281 - loss: 0.6904 - val_accuracy: 0.5326 - val_loss: 0.6899
Epoch 5/20
4904/4904 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.5361 - loss: 0.6892 - val_accuracy: 0.5450 - val_loss: 0.6872
Epoch 6/20
4904/4904 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.5438 - loss: 0.6869 - val_accuracy: 0.5500 - val_loss: 0.6851
Epoch 7/20
4904/4904 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.5493 - loss: 0.6853 - val_accuracy: 0.5533 - val_loss: 0.6826
Epoch 8/20
4904/4904 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.5542 - loss: 0

## Evaluate the Model

In [16]:
# Get the predictions
y_pred_prob = model.predict(X_test)

# Convert probabilities to binary (0 or 1) using a threshold
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate F1 Score, Precision and Recall value
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

2452/2452 ━━━━━━━━━━━━━━━━━━━━ 2s 882us/step
F1 Score: 0.5426
Precision: 0.5682
Recall: 0.5193
